## Fixed-Income Pricing Model Delovpment And Validation 

### Pricing Model: DCF | Asset Class: Fixed Income | Finacial Product: US Treasury Bond

#### Import Liabaries

In [3]:
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [5]:
dayCountMapping = {'1 Mo': 30, '2 Mo': 60, '3 Mo': 90, '4 Mo': 120, '6 Mo': 180, '1 Yr': 360, 
                   '2 Yr': 720, '3 Yr': 1080, '5 Yr': 1800, '7 Yr': 2520, '10 Yr': 3600, '20 Yr': 7200, '30 Yr': 10800}

#### Instrument Trade Data

In [10]:
trade_data = pd.read_csv("C:\\Users\\dines\\Desktop\\Market Risk Project\\10Y US Treasury Note_91282CJZ5\\Script 91282CJZ5.csv",index_col="CUSIP")

In [12]:
trade_data

,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price per $100
CUSIP,,,,,,
91282CJZ5,Note,9-Year 11-Month,03/12/2024,03/15/2024,02/15/2034,98.656761
91282CJZ5,Note,10-Year,02/07/2024,02/15/2024,02/15/2034,99.243059


#### Insturment Features

In [83]:
CUSIP=trade_data.index[-1]
faceValue = 100
cuponRate = 0.04
maturityPeriod = 10
frequency = 2
auctionDate = pd.to_datetime(trade_data['Auction Date'].values[-1]).date()
issueDate = pd.to_datetime(trade_data['Issue Date'].values[-1]).date()
maturityDate = pd.to_datetime(trade_data['Maturity Date'].values[-1]).date()
issuePrice = trade_data['Price per $100'].values[-1]

In [30]:
valuationDate = auctionDate

#### Coupon Dates

In [33]:
Y, M, D = issueDate.year, issueDate.month, issueDate.day

cuponDates = []
for i in range(1,(maturityPeriod*frequency) + 1):
    if i % 2 != 0:
        cuponDates.append(date(Y, M+6, D))
    else:
        cuponDates.append(date(Y+1, M, D))

print(cuponDates)

[datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15), datetime.date(2024, 8, 15), datetime.date(2025, 2, 15)]


In [41]:
cuponDays = [180 * i for i in range(1,(maturityPeriod*frequency) + 1)]
print(cuponDays)

[180, 360, 540, 720, 900, 1080, 1260, 1440, 1620, 1800, 1980, 2160, 2340, 2520, 2700, 2880, 3060, 3240, 3420, 3600]


## Market Data Preparation

#### Intrest Rate

In [45]:
class InterestRates:

    def __init__(self, Y):
        self.Y = Y

    def USTreasury(self, ratesCurve=False, COBDate=None):
        USTreasuryData = pd.read_csv(f'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/{self.Y}/all?type=daily_treasury_yield_curve&field_tdr_date_value={self.Y}&page&_format=csv', index_col='Date').iloc[::-1]  
        USTreasuryData.index = pd.to_datetime(USTreasuryData.index)
        
        if ratesCurve == True:
            if COBDate == None:
                return USTreasuryData[USTreasuryData.index == USTreasuryData.index.max()]
            else:
                return USTreasuryData[USTreasuryData.index == COBDate]
        else:
            return USTreasuryData

In [51]:
spotRates = InterestRates(Y=valuationDate.year).USTreasury(ratesCurve=True, COBDate=pd.to_datetime(valuationDate))
spotRates = pd.DataFrame(data=spotRates.values, index=spotRates.index, columns=dayCountMapping.values())

In [53]:
spotRates

,30,60,90,120,180,360,720,1080,1800,2520,3600,7200,10800
Date,,,,,,,,,,,,,
2024-02-07,5.47,5.49,5.43,5.4,5.23,4.83,4.41,4.16,4.06,4.09,4.09,4.41,4.31


####  Interst Rate Determination

In [56]:
tenors , interestRates = spotRates.columns/360 , spotRates.values[0]/100
print(tenors, interestRates)

Index([0.08333333333333333, 0.16666666666666666,                0.25,
        0.3333333333333333,                 0.5,                 1.0,
                       2.0,                 3.0,                 5.0,
                       7.0,                10.0,                20.0,
                      30.0],
      dtype='float64') [0.0547 0.0549 0.0543 0.054  0.0523 0.0483 0.0441 0.0416 0.0406 0.0409
 0.0409 0.0441 0.0431]


In [62]:
interpfunction = interpolate.CubicSpline(x= tenors, y= interestRates)
tenorsInterp = [t/360 for t in cuponDays]
interpolatedRates = interpfunction(tenorsInterp)
print(interpolatedRates)

[0.0523     0.0483     0.04590211 0.0441     0.04266305 0.0416
 0.04095041 0.04063765 0.04055607 0.0406     0.04067914 0.04076456
 0.0408427  0.0409     0.04092684 0.04092941 0.04091787 0.04090235
 0.04089301 0.0409    ]


### Pricing Model

#### Discount Cashflow (DCF) Approach

In [68]:
futurevalueCF = [faceValue*(cuponRate/frequency) for t in range(1,(maturityPeriod*frequency)+1)]
futurevalueCF[-1] += faceValue
print(futurevalueCF)

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 102.0]


In [72]:
annualizedRate = [(1+i/frequency)**frequency -1 for i in interpolatedRates]
print(annualizedRate)

[0.05298382249999989, 0.04888322249999977, 0.046428859862901906, 0.044586202499999894, 0.04311808149863272, 0.042032639999999954, 0.041369640831615984, 0.04105050425503154, 0.04096726619813151, 0.041012089999999946, 0.04109283336061953, 0.04117999539754358, 0.04125973337257505, 0.04131820250000007, 0.041345590031032, 0.041348216341630106, 0.04133643600516024, 0.041320603854411164, 0.04131107452273519, 0.04131820250000007]


In [76]:
presentValueCF = [futurevalueCF[i]/(1+interpolatedRates[i]/frequency) ** (i+1) for i in range(maturityPeriod*frequency)]
bondPrice = sum(presentValueCF)
print(bondPrice,presentValueCF)

99.21523506308216 [1.9490327924767337, 1.9067899620255395, 1.8683809586480165, 1.832911201818863, 1.7996842326717282, 1.767608280580518, 1.7354527820962062, 1.7027182866762691, 1.669410600034037, 1.6358788184233342, 1.6026473976572733, 1.569911259250721, 1.5377863942761938, 1.506418967985966, 1.4759389001919925, 1.446312620053649, 1.4174441231694133, 1.3892162085359026, 1.3614928569001636, 68.04019841960964]


#### Model Error

In [79]:
print('Model Price:',bondPrice)
print('Market Price:',issuePrice)

priceDiff = bondPrice-issuePrice
print('Price Difference:', round(priceDiff,4))

Model Price: 99.21523506308216
Market Price: 99.243059
Price Difference: -0.0278


In [81]:
pricingResult =pd.Series({"Model Price":bondPrice,"Issue Price":issuePrice,"Price Difference":priceDiff})

In [85]:
pricingResult.to_csv(f"C:\\Users\\dines\\Desktop\\Market Risk Project\\{maturityPeriod}Y US Treasury Note_{CUSIP}\\Pricing RESULTS for {CUSIP}.csv")